In [1]:
import json
from sentence_transformers import SentenceTransformer, util
import torch
import numpy as np
import random
from tqdm.notebook import tqdm
# import os
import inflect
%config Completer.use_jedi = False

In [2]:
def process(s):
    return s.replace('"', '').replace("'", '').replace("[","").replace("]","")

def get_accuracy(caption_embeddings, class_embeddings, gt_label, topk=5):
    similarity_matrix = util.cos_sim(caption_embeddings, class_embeddings)
    pseudoLabel = similarity_matrix.topk(topk, 1).indices    
    gt_label = torch.Tensor(gt_label).view(-1,1).repeat(1,topk)
    matched_labels = (torch.Tensor(pseudoLabel) == torch.Tensor(gt_label))
    top1_acc = matched_labels[:,0].sum()/len(gt_label)
    topk_acc = matched_labels.any(1).sum()/len(gt_label)
    return top1_acc, topk_acc

In [3]:
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1').cuda()
model = model.eval();

In [14]:
dataset = "domainnet"
data = json.load(open(f"../metadata/{dataset}.json"))

cats = {int(c["category_id"]):c["category_name"] for c in data["categories"]}
cats = [cats[idx].replace("+"," ").replace("_"," ").replace("-"," ") for idx in range(345)]
class_embeddings = model.encode(cats)

category_to_id = {c["category_name"].lower():c["category_id"] for c in data["categories"]}
id_to_category = {int(v):k for k,v in category_to_id.items()}


source = "real"
target = "real"
data = data[f'{target}_train']
if "ego" not in dataset:
    id_to_label = {seg.get("segment_id",seg["image_id"]):seg["category"] for seg in data["annotations"]}
#     id_to_caption = {metadata.get("segment_id",metadata["image_id"]):" ".join([metadata["caption"], metadata["tags"].replace(","," "), metadata["description"]]) for metadata in data["metadata"]}
    id_to_caption = {metadata["image_id"]:metadata["blip2_cap"] for metadata in data["metadata"]}
else:
    id_to_label = {seg.get("segment_id"):seg["category"] for seg in data["annotations"]}
    id_to_caption = {metadata.get("segment_id"):metadata["text_caption"] for metadata in data["descriptions"]}

In [16]:
llm_label = json.load(open(f"../llm_labels/llm_label_{dataset}_{source}_{target}_7b_noICL.json"))["labels"]
id_to_llm = {c['image_id']:process(c['label_name']) for c in llm_label}

all_ids = list(id_to_llm.keys())

In [17]:
all_labels = [id_to_label[idx] for idx in all_ids]
all_predicted_names = [id_to_llm[idx] for idx in all_ids]

In [18]:
predicted_embeddings = model.encode(all_predicted_names)

In [19]:
get_accuracy(predicted_embeddings, class_embeddings, all_labels)

(tensor(0.3791), tensor(0.5661))

In [21]:
all_captions = [id_to_caption[idx] for idx in all_ids]

the aircraft carrier is docked in the water aircraft_carrier
a military aircraft is parked on the flight deck of an aircraft carrier aircraft_carrier
the us navy's littoral combat ship person, left, and the us navy's litt ship
us navy ships in the ocean with a large aircraft carrier aircraft_carrier
an aircraft is flying over the aircraft carrier aircraft_carrier
the uss person is seen in the water aircraft_carrier
the chinese navy's first aircraft carrier, the chinese aircraft carrier china's first aircraft carrier aircraft_carrier
the aircraft carrier hms person is seen in the middle of the ocean aircraft_carrier
the aircraft carrier is sailing in the ocean aircraft_carrier
the aircraft carrier is being built in the water aircraft_carrier
an aircraft carrier is being hit by smoke aircraft_carrier
the indian navy's indra class aircraft carrier indra aircraft_carrier
an aircraft carrier in the open ocean aircraft_carrier
the aircraft carrier is sailing in the ocean aircraft_carrier
an 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [23]:
for cap, name,lbl in zip(all_captions[-100:], all_predicted_names[-100:], all_labels[-100:]):
    print(cap + "--" + name + "--" + id_to_category[lbl])
    print()

the forex chart shows the price action and the price action--chart--zigzag

the pattern for the crochet hook is shown--object--zigzag

a building with many windows on the side of it--building--zigzag

a black and white pattern with many small black and white shapes--pattern--zigzag

a black and white chevron pattern background--pattern--zigzag

a blue and white chevron pattern--pattern--zigzag

a pink box with a chevron pattern on it--box--zigzag

a quilt with colorful squares on it--quilt--zigzag

a colorful pattern with a diagonal design--alarm_clock--zigzag

a colorful zigzag pattern with a rainbow--pattern--zigzag

a blue and white chevron pattern--pattern--zigzag

a close up of a metal and orange pulley--machine--zigzag

a blue and gray striped pattern with a diagonal line--alarm_clock--zigzag

a person is sewing a piece of fabric with a sewing machine--sewing_machine--zigzag

a black and white star stamp on a white background--stamp--zigzag

a barbell with a long handle on a whit